In [1]:
import pandas as pd

file_to_load = "Resources/nba_2017_salary.csv"

salary = pd.read_csv(file_to_load)
salary.head()

,NAME,POSITION,TEAM,SALARY
0,LeBron James,SF,Cleveland Cavaliers,30963450.0
1,Mike Conley,PG,Memphis Grizzlies,26540100.0
2,Al Horford,C,Boston Celtics,26540100.0
3,Dirk Nowitzki,PF,Dallas Mavericks,25000000.0
4,Carmelo Anthony,SF,New York Knicks,24559380.0


In [7]:
team_salary_avg = salary.groupby(["TEAM"]).mean()["SALARY"].rename("Average Salary")
new_df = pd.DataFrame(team_salary_avg)
new_df.style.format({"Average Salary":"${:,.2f}"})
new_df

,Average Salary
TEAM,
Atlanta Hawks,4.917651e+06
Boston Celtics,5.050278e+06
Brooklyn Nets,4.435575e+06
Charlotte Hornets,5.318635e+06
Chicago Bulls,6.620476e+06
Cleveland Cavaliers,1.056409e+07
Dallas Mavericks,6.697628e+06
Denver Nuggets,4.645013e+06
Detroit Pistons,6.574762e+06


In [8]:
import requests
import pymongo
from bs4 import BeautifulSoup as bs

In [39]:
url = "https://www.basketball-reference.com/leagues/NBA_2017_per_minute.html"
page = requests.get(url)
page

<Response [200]>

In [41]:
soup = bs(page.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="no-js" data-root="/home/bbr/build" data-version="klecko-" itemscope="" itemtype="https://schema.org/WebSite" lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="ie=edge" http-equiv="x-ua-compatible"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport">
   <link href="https://d2p3bygnnzw9w3.cloudfront.net/req/202103121" rel="dns-prefetch"/>
   <!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->
   <script async="true" type="text/javascript">
    (function() {
	var host = window.location.hostname;
	var element = document.createElement('script');
	var firstScript = document.getElementsByTagName('script')[0];
	var url = 'https://quantcast.mgr.consensu.org'
	    .concat('/choice/', 'XwNYEpNeFfhfr', '/', host, '/choice.js')
	var uspTries = 0;
	var uspTriesLimit = 3;
	element.async = true;
	element.type = 'text/javascript';
	element.src = url;
	
	firstScript.parentNode.insertBefore(element, firstScrip

In [15]:
read_table = pd.read_html(url)
read_table

[      Rk             Player Pos Age   Tm   G  GS    MP   FG   FGA  ...   FT%  \
 0      1       Álex Abrines  SG  23  OKC  68   6  1055  4.6  11.6  ...  .898   
 1      2         Quincy Acy  PF  26  TOT  38   1   558  4.5  11.0  ...  .750   
 2      2         Quincy Acy  PF  26  DAL   6   0    48  3.7  12.7  ...  .667   
 3      2         Quincy Acy  PF  26  BRK  32   1   510  4.6  10.8  ...  .754   
 4      3       Steven Adams   C  23  OKC  80  80  2389  5.6   9.9  ...  .611   
 ..   ...                ...  ..  ..  ...  ..  ..   ...  ...   ...  ...   ...   
 614  482        Cody Zeller   C  24  CHO  62  58  1725  5.3   9.2  ...  .679   
 615  483       Tyler Zeller   C  27  BOS  51   5   525  5.3  10.8  ...  .564   
 616  484  Stephen Zimmerman   C  20  ORL  19   0   108  3.3  10.3  ...  .600   
 617  485        Paul Zipser  SF  22  CHI  44  18   843  3.8   9.4  ...  .775   
 618  486        Ivica Zubac   C  19  LAL  38  11   609  7.4  14.1  ...  .653   
 
      ORB  DRB   TRB  AST 

In [25]:
type(read_table)

list

In [26]:
df = read_table[0]
df.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Álex Abrines,SG,23,OKC,68,6,1055,4.6,11.6,...,.898,0.6,2.3,2.9,1.4,1.3,0.3,1.1,3.9,13.9
1,2,Quincy Acy,PF,26,TOT,38,1,558,4.5,11.0,...,.750,1.3,6.1,7.4,1.2,0.9,1.0,1.4,4.3,14.3
2,2,Quincy Acy,PF,26,DAL,6,0,48,3.7,12.7,...,.667,1.5,4.5,6.0,0.0,0.0,0.0,1.5,6.8,9.7
3,2,Quincy Acy,PF,26,BRK,32,1,510,4.6,10.8,...,.754,1.3,6.3,7.6,1.3,1.0,1.1,1.3,4.1,14.8
4,3,Steven Adams,C,23,OKC,80,80,2389,5.6,9.9,...,.611,4.2,5.0,9.2,1.3,1.3,1.2,2.2,2.9,13.6


In [33]:
df = df.reset_index(drop=True)
df.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Álex Abrines,SG,23,OKC,68,6,1055,4.6,11.6,...,.898,0.6,2.3,2.9,1.4,1.3,0.3,1.1,3.9,13.9
1,2,Quincy Acy,PF,26,TOT,38,1,558,4.5,11.0,...,.750,1.3,6.1,7.4,1.2,0.9,1.0,1.4,4.3,14.3
2,2,Quincy Acy,PF,26,DAL,6,0,48,3.7,12.7,...,.667,1.5,4.5,6.0,0.0,0.0,0.0,1.5,6.8,9.7
3,2,Quincy Acy,PF,26,BRK,32,1,510,4.6,10.8,...,.754,1.3,6.3,7.6,1.3,1.0,1.1,1.3,4.1,14.8
4,3,Steven Adams,C,23,OKC,80,80,2389,5.6,9.9,...,.611,4.2,5.0,9.2,1.3,1.3,1.2,2.2,2.9,13.6


In [38]:
df.dtypes

Rk        object
Player    object
Pos       object
Age       object
Tm        object
G         object
GS        object
MP        object
FG        object
FGA       object
FG%       object
3P        object
3PA       object
3P%       object
2P        object
2PA       object
2P%       object
FT        object
FTA       object
FT%       object
ORB       object
DRB       object
TRB       object
AST       object
STL       object
BLK       object
TOV       object
PF        object
PTS       object
dtype: object

In [37]:
df["Age"] = df["Age"].str.replace(',','.').astype('int')

ValueError: invalid literal for int() with base 10: 'Age'

In [34]:
age_avg = df.groupby(["Tm"]).mean()["Age"].rename("Average Player Age")
age_df = pd.DataFrame(age_avg)
age_df.head()

DataError: No numeric types to aggregate